# Python Basecamp

### Karthikeyan Sankaran, 17th June, 2018

### End to End ML Pipeline [BigMart Sales]: Notebook 3 - Feature Selection

In [90]:
import pandas as pd
import numpy as np

In [91]:
%store -r train_feats
%store -r target_df

In [92]:
train_for_feature_selection = train_feats.copy()
train_for_feature_selection.head()

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,...,ClusterName_lb_enc,Item_Identifier_freq,Outlet_Identifier_freq,Outlet_Type_freq,Item_MRP_clipped_tf,Item_Visibility_Imputed_tf,Item_Weight_Imputed_tf,pca_1,pca_2,pca_3
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,...,0,8,930,5577,1.754051,0.015920,2.332144,1.741497,-0.759029,-0.780953
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,...,1,6,928,928,-1.494959,0.019095,1.934416,-1.499466,0.274907,-0.781373
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,...,2,7,930,5577,0.009910,0.016621,2.917771,0.005386,-0.724442,-0.781842
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,19.20,1998,OUT010,Tier 3,...,2,6,555,1083,0.662436,0.017677,3.005683,0.671598,0.268986,0.217743
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,8.93,1987,OUT013,Tier 3,...,1,6,932,5577,-1.404807,0.009732,2.295560,-1.392042,1.281675,1.217797


In [93]:
target_df.head()

,Item_Outlet_Sales,Unit_Sales,Item_Outlet_Sales_log,Unit_Sales_log
0,3735.1380,15.0,8.225808,2.772589
1,443.4228,9.0,6.096776,2.302585
2,2097.2700,15.0,7.648868,2.772589
3,732.3800,4.0,6.597664,1.609438
4,994.7052,18.0,6.903451,2.944439


In [94]:
train_for_feature_selection.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 62 columns):
Item_Fat_Content                                 8523 non-null object
Item_Identifier                                  8523 non-null object
Item_MRP                                         8523 non-null float64
Item_Outlet_Sales                                8523 non-null float64
Item_Type                                        8523 non-null object
Item_Visibility                                  8523 non-null float64
Item_Weight                                      7060 non-null float64
Outlet_Establishment_Year                        8523 non-null int64
Outlet_Identifier                                8523 non-null object
Outlet_Location_Type                             8523 non-null object
Outlet_Size                                      6113 non-null object
Outlet_Type                                      8523 non-null object
source                                           8

In [95]:
features = ['Outlet_Size_Imputed_lb_enc','Outlet_Size_FillNA_lb_enc','Outlet_Size_NaN_lb_enc',
            'Item_Weight_NaN_lb_enc','Item_Visibility_NaN_lb_enc','Item_Fat_Content_modified_lb_enc','Item_Type_Combined_lb_enc',
            'PriceLevels_lb_enc','WeightLevels_lb_enc','Item_Identifier_freq','Outlet_Identifier_freq','Outlet_Type_freq',
            'Item_Identifier_mean_enc_fold','Outlet_Identifier_mean_enc_fold',
            'Outlet_Type:T.Supermarket Type1::Outlet_Years','Outlet_Years:Item_MRP_clipped']

new_train_df = train_for_feature_selection[features]

In [96]:
x_train = new_train_df.values
y_train = target_df['Item_Outlet_Sales'].values

In [97]:
n_samples, n_features = x_train.shape
# perform evaluation on classification task
num_fea = 8    # number of selected features

In [98]:
features_df = pd.DataFrame()

In [99]:
# Many different features selection methods are available - http://featureselection.asu.edu/
# Refer to examples at https://github.com/jundongl/scikit-feature/tree/master/skfeature/example
selection_methods=['MRMR','CIFE','CMIM','MIM','MIFS','ICAP','JMI','DISR']

In [100]:
if 'MRMR' in selection_methods:
    from skfeature.function.information_theoretical_based import MRMR

    idx_mrmr = MRMR.mrmr(x_train, y_train, n_selected_features=num_fea)

    temp_features_df = pd.DataFrame(new_train_df.columns[idx_mrmr])
    temp_features_df.columns = ['MRMR']
    features_df = pd.concat([features_df,temp_features_df],axis=1)

In [101]:
if 'CIFE' in selection_methods:
    from skfeature.function.information_theoretical_based import CIFE

    idx_cife = CIFE.cife(x_train, y_train, n_selected_features=num_fea)

    temp_features_df = pd.DataFrame(new_train_df.columns[idx_cife])
    temp_features_df.columns = ['CIFE']
    features_df = pd.concat([features_df,temp_features_df],axis=1)

In [102]:
if 'CMIM' in selection_methods:
    from skfeature.function.information_theoretical_based import CMIM

    idx_cmim = CMIM.cmim(x_train, y_train, n_selected_features=num_fea)

    temp_features_df = pd.DataFrame(new_train_df.columns[idx_cmim])
    temp_features_df.columns = ['CMIM']
    features_df = pd.concat([features_df,temp_features_df],axis=1)

In [103]:
if 'MIM' in selection_methods:
    from skfeature.function.information_theoretical_based import MIM

    idx_mim = MIM.mim(x_train, y_train, n_selected_features=num_fea)

    temp_features_df = pd.DataFrame(new_train_df.columns[idx_mim])
    temp_features_df.columns = ['MIM']
    features_df = pd.concat([features_df,temp_features_df],axis=1)

In [104]:
if 'MIFS' in selection_methods:
    from skfeature.function.information_theoretical_based import MIFS

    idx_mifs = MIFS.mifs(x_train, y_train, n_selected_features=num_fea)

    temp_features_df = pd.DataFrame(new_train_df.columns[idx_mifs])
    temp_features_df.columns = ['MIFS']
    features_df = pd.concat([features_df,temp_features_df],axis=1)

In [105]:
if 'ICAP' in selection_methods:
    from skfeature.function.information_theoretical_based import ICAP

    idx_icap = ICAP.icap(x_train, y_train, n_selected_features=num_fea)

    temp_features_df = pd.DataFrame(new_train_df.columns[idx_icap])
    temp_features_df.columns = ['ICAP']
    features_df = pd.concat([features_df,temp_features_df],axis=1)

In [106]:
if 'JMI' in selection_methods:
    from skfeature.function.information_theoretical_based import JMI

    idx_jmi = JMI.jmi(x_train, y_train, n_selected_features=num_fea)

    temp_features_df = pd.DataFrame(new_train_df.columns[idx_jmi])
    temp_features_df.columns = ['JMI']
    features_df = pd.concat([features_df,temp_features_df],axis=1)

In [107]:
if 'DISR' in selection_methods:
    from skfeature.function.information_theoretical_based import DISR

    idx_disr = DISR.disr(x_train, y_train, n_selected_features=num_fea)

    temp_features_df = pd.DataFrame(new_train_df.columns[idx_disr])
    temp_features_df.columns = ['DISR']
    features_df = pd.concat([features_df,temp_features_df],axis=1)

In [108]:
features_df

,MRMR,CIFE,CMIM,MIM,MIFS,ICAP,JMI,DISR
0,Outlet_Years:Item_MRP_clipped,Outlet_Years:Item_MRP_clipped,Outlet_Years:Item_MRP_clipped,Outlet_Years:Item_MRP_clipped,Outlet_Years:Item_MRP_clipped,Outlet_Years:Item_MRP_clipped,Outlet_Years:Item_MRP_clipped,Outlet_Years:Item_MRP_clipped
1,Item_Visibility_NaN_lb_enc,Item_Identifier_mean_enc_fold,Item_Identifier_mean_enc_fold,Item_Identifier_mean_enc_fold,Item_Identifier_mean_enc_fold,Item_Identifier_mean_enc_fold,Item_Identifier_mean_enc_fold,Item_Identifier_mean_enc_fold
2,Item_Identifier_mean_enc_fold,Outlet_Size_FillNA_lb_enc,WeightLevels_lb_enc,Outlet_Identifier_mean_enc_fold,Outlet_Type_freq,WeightLevels_lb_enc,Outlet_Identifier_mean_enc_fold,Outlet_Identifier_mean_enc_fold
3,Outlet_Identifier_mean_enc_fold,Item_Weight_NaN_lb_enc,Item_Identifier_freq,PriceLevels_lb_enc,Item_Visibility_NaN_lb_enc,Item_Identifier_freq,Item_Identifier_freq,Item_Identifier_freq
4,PriceLevels_lb_enc,Item_Visibility_NaN_lb_enc,Item_Fat_Content_modified_lb_enc,Outlet_Identifier_freq,Outlet_Size_NaN_lb_enc,Item_Fat_Content_modified_lb_enc,WeightLevels_lb_enc,WeightLevels_lb_enc
5,Item_Identifier_freq,Item_Fat_Content_modified_lb_enc,Outlet_Identifier_mean_enc_fold,Item_Identifier_freq,Item_Weight_NaN_lb_enc,Outlet_Identifier_mean_enc_fold,PriceLevels_lb_enc,PriceLevels_lb_enc
6,WeightLevels_lb_enc,Outlet_Size_NaN_lb_enc,Item_Type_Combined_lb_enc,Outlet_Type:T.Supermarket Type1::Outlet_Years,Item_Type_Combined_lb_enc,Item_Type_Combined_lb_enc,Outlet_Identifier_freq,Outlet_Identifier_freq
7,Outlet_Identifier_freq,Item_Identifier_freq,Item_Visibility_NaN_lb_enc,WeightLevels_lb_enc,PriceLevels_lb_enc,Item_Visibility_NaN_lb_enc,Outlet_Type:T.Supermarket Type1::Outlet_Years,Outlet_Type:T.Supermarket Type1::Outlet_Years
